In [ ]:
!pip install transformers==4.12.5

In [ ]:
import torch
#from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#import torch.nn.functional as F
from transformers import BertTokenizer #BertConfig,AdamW, 

import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split


#to_preprocessing
import re
import string
import nltk
from nltk.corpus import stopwords

#from tqdm import tqdm, trange,tnrange,tqdm_notebook
#import random
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import io
#%matplotlib inline
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# Let me know if this code makes sense or if y'all have any questions! 
# This is my first time doing data processing with this kind of data so there might be errors

In [ ]:
# identify and specify the GPU as the device, later in training loop we will load data into device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

Load Data

In [ ]:
# importing the dataset 
df_train = pd.read_csv("/kaggle/input/emotions-dataset-for-nlp/train.txt", delimiter=';', header=None, names=['Text','Sentiment'])
df_test = pd.read_csv("/kaggle/input/emotions-dataset-for-nlp/test.txt", delimiter=';', header=None, names=['Text','Sentiment'])
df_val = pd.read_csv("/kaggle/input/emotions-dataset-for-nlp/val.txt", delimiter=';', header=None, names=['Text','Sentiment'])
df = pd.concat([df_train, df_val, df_test], axis = 0)

: 

Preproccessing Data

In [ ]:
def decontracted(phrase):
    """
    We first define a function to expand the contracted phrase into normal words
    """
        
    phrase = re.sub(r"wont", "will not", phrase)
    phrase = re.sub(r"wouldnt", "would not", phrase)
    phrase = re.sub(r"shouldnt", "should not", phrase)
    phrase = re.sub(r"couldnt", "could not", phrase)
    phrase = re.sub(r"cudnt", "could not", phrase)
    phrase = re.sub(r"cant", "can not", phrase)
    phrase = re.sub(r"dont", "do not", phrase)
    phrase = re.sub(r"doesnt", "does not", phrase)
    phrase = re.sub(r"didnt", "did not", phrase)
    phrase = re.sub(r"wasnt", "was not", phrase)
    phrase = re.sub(r"werent", "were not", phrase)
    phrase = re.sub(r"havent", "have not", phrase)
    phrase = re.sub(r"hadnt", "had not", phrase)
    phrase = re.sub(r"neednt", "need not", phrase)
    phrase = re.sub(r"isnt", "is not", phrase)
    phrase = re.sub(r"arent", "are not", phrase)
    phrase = re.sub(r"hasnt", "are not", phrase)
    
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
# Get rid of user handles, tags, link, punctuation
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = decontracted(text)
    return text

In [ ]:
# Apple cleaning function for the dataframe
df["Text"] = df["Text"].apply(lambda x: clean_text(x))

In [ ]:
# Transform emotion label into encoded label
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['label_enc'] = labelencoder.fit_transform(df['Sentiment'])
df.rename(columns={'label':'label_desc'},inplace=True)
df.rename(columns={'label_enc':'label'},inplace=True)

In [ ]:
# Split dataset
train_text, val_text, train_labels, val_labels = train_test_split(df['Text'], df['label'], random_state=42, test_size=0.1, stratify=df['label'])

Tokenizer

In [ ]:
MAX_LEN = 256
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", lower = True)

In [ ]:
# tokenize and encode sequences in the training set (for Bertbase fine tuning)
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = MAX_LEN,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = MAX_LEN,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)